In [126]:
from helpers import *

In [127]:
nodes, costs, D = read_data(filename="TSPA.csv")

In [128]:
# Heuristic - randomized selection of nodes with probability guided by node cost + length of 2 neighbouring edges
# returns array of costs - the higher the better
def heuristic_value(path, D, costs):
    scores = np.array(costs)
    for idx, node in enumerate(path):
        prev_node = path[(idx - 1) % len(path)]
        next_node = path[(idx + 1) % len(path)]
        scores[idx] += D[prev_node, node] + D[node, next_node]
    return np.array(scores)


# remove n separate subpaths of total length x% of full path, chosen points are middle of subpath
def destroy(path, D, costs, size=0.25, num_subpaths=5):
    path_len = len(path)
    N = int(path_len*size)
    subpath_len = N // num_subpaths
    free_nodes = np.full(path_len, True, dtype=bool)
    scores = heuristic_value(path, D, costs)
    scores = scores[np.array(path)]
    p = scores / scores.sum()
    removed_nodes = []
    for _ in range(num_subpaths):
        choice = None
        while choice is None:
            idx = np.random.choice(path_len, p=p)
            if free_nodes[idx]:
                choice = idx
        for i in range(0, subpath_len//2 + 1):
            idx1, idx2 = (choice - i) % path_len, (choice + i) % path_len
            removed_nodes.append(path[idx1])
            removed_nodes.append(path[idx2])
            free_nodes[idx1] = False
            free_nodes[idx2] = False

    destroyed_path = np.array(path)
    destroyed_path = destroyed_path[free_nodes]
    removed_nodes = list(np.unique(removed_nodes))

    return destroyed_path, removed_nodes

def greedy_regret_repair(path, nodes_available, D, costs, weights=0.5, closed_set=True):
    if not closed_set:
        not_in_path = [x for x in list(range(D.shape[0])) if x not in path]
        for x in range(D.shape[0]):
            if np.random.choice([0,1]):
                nodes_available.append(not_in_path[x])
    target_length = math.ceil(len(D) / 2)
    path = list(path) + [path[0]]
    edges = []
    for idx, node in enumerate(path):
        if idx == len(path)-1: break
        edges.append([node, path[idx+1]])
    while len(path) < target_length+1:
        M = np.zeros((len(nodes_available), len(edges)))
        indices = np.array(nodes_available)
        for edge_ix in range(len(edges)):
            a, b = edges[edge_ix]
            var = D[a, :] + D[:, b] - D[a, b] + costs
            M[:,edge_ix] = var[indices]
        best_score = -MAX_DIST
        replaced_edge = 0
        best_node = 0
        for node_idx in range(len(nodes_available)):
            best, second_best = np.partition(M[node_idx], 1)[:2]
            regret = second_best - best
            score = weights * regret - (1 - weights) * np.min(M[node_idx])
            if score > best_score:
                best_score = score
                replaced_edge = np.argmin(M[node_idx])
                best_node = nodes_available[node_idx]
        path.insert(replaced_edge+1, best_node)
        nodes_available.remove(best_node)
        a, b = edges[replaced_edge]
        edges.pop(replaced_edge)
        edges.insert(replaced_edge, [a, best_node])
        edges.insert(replaced_edge + 1, [best_node, b])

    return np.array(path[:-1])


def LSNS(D, costs, random_start=True, LS_enable=False, time_limit=10, closed_set=True, num_subpaths=5):
    time_start = time.time()
    best_score = MAX_DIST

    if random_start:
        current_path = random_sequence(D)
        current_path = local_search_deltas(current_path, costs, D)
        score = evaluate(D, current_path, costs)
    else:
        current_path = greedy_regret(D, costs, start_node=random.randint(0, len(D)-1))

    while True:
        destroyed_path, removed_nodes = destroy(current_path, D, costs)
        path = greedy_regret_repair(destroyed_path, removed_nodes, D, costs, closed_set)

        if LS_enable:
            path = local_search_deltas(path, costs, D)

        score = evaluate(D, path, costs)
        if score < best_score:
            current_path = path
            best_score = score

        time_running = time.time() - time_start
        if time_running > time_limit:
            break

    return current_path


In [130]:
path = LSNS(D, costs, random_start=False, LS_enable=False, closed_set=True, time_limit=30, num_subpaths=5)
score = evaluate(D, path, costs)
print(f'Path: {path}\nScore: {score}')

Path: [178  19   0 149  50  43  77   4 114 121  91 161  76 145  40 128 132  36
  55  22 117 171 194  79  21 170 186 127  88 153 175 192 150 199  41 177
   1  75 189 109 119 130 152  11  48 106  26   8 169  95 124  80  14 111
  31  73  89  94  72 190  98 156   6  66 112   5  51 135  99 101 167  45
  24 141  87 144 102 154  81 108  62  53 195 113  74 163  61  71  20  64
 185  70  96  27 116 147  59 143 159 164]
Score: 75386
